In [1]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

### 生データの取得
- 漏れを0で埋める

In [2]:
def read_raw_data(f_path):
    df = pd.read_excel(f_path, header=0, index_col=0)
    df = df.sort_index()
    df.fillna(0, inplace=True)
    return df

### データの前処理
- 広告コード抽出
- 広告コード情報のコラム（定数コラム）を削除
- Salesの合計を計算
- 一週間の移動合計を取得

In [3]:
def _sales_sum(row):
    return sum(v for i, v in row.items() if 'Sales' in i)

def preprocess_raw_data(df):
    
    # Sales項を合計
    df['Sales'] = df.apply(lambda x: _sales_sum(x), axis=1)
    
    # 七日間の移動合計
    df['SalesSum'] = df['Sales'].rolling('7D').sum()
    df['ImpressionsSum'] = df['Impressions'].rolling('7D').sum()
    df['ClicksSum'] = df['Clicks'].rolling('7D').sum()
    df['CostSum'] = df['Cost'].rolling('7D').sum()
    
    
    code = {}
    for c in ['MarketId', 'AdvertiserId', 'CampaignId', 'CategoryId']:
        code[c] = df.iloc[0][c]

        
    df_x = df.drop(columns=['Market', 'MarketId', 'AdvertiserId', 'AdvertiserName', 
                          'CampaignId', 'CampaignName', 'CategoryId', 'CategoryName'])

    return df_x, df, code

### 学習データ生成のアルゴリズム
- 現在日から遡る７日分のデータから一週間後を予測
- 現在日から一週間を間隔に遡る５週間分のデータから一週間後を予測

In [4]:
def create_train_data_consistent(df_x, df_y):
    X, Y = [], []
    for i, r in df_x.iterrows():
        if all(i + pd.DateOffset(d) in df_x.index and df_x.loc[i + pd.DateOffset(d)]['CpcBid'] > 0
               for d in [0,1,2,3,4,5,6,13]):
            
            x = pd.concat([df_x.loc[i + pd.DateOffset(d)].rename(lambda x: f'{x}_{6-d}') 
                           for d in range(6, -1, -1)])
            y = df_y.loc[i + pd.DateOffset(13)]
            
            x.name = i + pd.DateOffset(6)
            x.loc['CpcBid_Y'] = y.loc['CpcBid']
            
            X += [x]
            Y += [y]
    
    if not X:
        return None, None
    
    df_X = pd.concat(X, axis=1, keys=[x.name for x in X]).T
    df_Y = pd.concat(Y, axis=1, keys=[y.name for y in Y]).T
    
    '''
    df_Y = df_Y[['Market', 'AdvertiserName', 'CampaignName', 'CategoryName', 
                 'SalesRent', 'SalesRyutsu', 'SalesSck', 'SalesMansionBkn', 
                 'SalesMansionKen', 'SalesKodateBkn', 'SalesKodateKen', 
                 'Sales', 'Impressions', 'Clicks', 'Cost',
                 'SalesSum', 'ImpressionsSum', 'ClicksSum', 'CostSum',
                ]]
    '''
    return df_X, df_Y

In [5]:
def create_train_data_skip(df_x, df_y):
    X, Y = [], []
    for i, r in df_x.iterrows():
        if all(i + pd.DateOffset(d) in df_x.index and df_x.loc[i + pd.DateOffset(d)]['CpcBid'] > 0
               for d in range(0, 36, 7)):
            
            x = pd.concat([df_x.loc[i + pd.DateOffset(d)].rename(lambda x: f'{x}_{(28-d)//7}') 
                           for d in range(28, -1, -7)])
            y = df_y.loc[i + pd.DateOffset(35)]
            
            x.name = i + pd.DateOffset(28)
            x.loc['CpcBid_Y'] = y.loc['CpcBid']
            
            X += [x]
            Y += [y]
    
    if not X:
        return None, None
    
    df_X = pd.concat(X, axis=1, keys=[x.name for x in X]).T
    df_Y = pd.concat(Y, axis=1, keys=[y.name for y in Y]).T
    '''
    df_Y = df_Y[['Market', 'AdvertiserName', 'CampaignName', 'CategoryName', 
                 'SalesRent', 'SalesRyutsu', 'SalesSck', 'SalesMansionBkn', 
                 'SalesMansionKen', 'SalesKodateBkn', 'SalesKodateKen', 
                 'Sales', 'Impressions', 'Clicks', 'Cost',
                 'SalesSum', 'ImpressionsSum', 'ClicksSum', 'CostSum',
                ]]
    '''
    return df_X, df_Y

### 学習データを作成

In [6]:
in_dir = './data/two_years_each/data/'
out_dir_cons = './data/two_years_each/data_consistent/'
out_dir_skip = './data/two_years_each/data_skip/'
for fi, f in enumerate(os.listdir(in_dir)):
    #if fi > 0: break
    
    print('-' * 25)
    print(f)
    
    f_name = os.path.splitext(f)[0]
    f_path = os.path.join(in_dir, f)
    
    df = read_raw_data(f_path)
    df_x, df_y, code = preprocess_raw_data(df)
    print(code)
    
    df_X, df_Y = create_train_data_consistent(df_x, df_y)
    if df_X is not None:
        for i, (k, v) in enumerate(code.items()): df_X.insert(loc=i, column=k, value=v)
        df_X.to_excel(os.path.join(out_dir_cons, f'{f_name}_X.xlsx'))
        df_Y.to_excel(os.path.join(out_dir_cons, f'{f_name}_Y.xlsx'))
        print(f'Number of consistent train data: {len(df_X)}')

    df_X, df_Y = create_train_data_skip(df_x, df_y)
    if df_X is not None:
        for i, (k, v) in enumerate(code.items()): df_X.insert(loc=i, column=k, value=v)
        df_X.to_excel(os.path.join(out_dir_skip, f'{f_name}_X.xlsx'))
        df_Y.to_excel(os.path.join(out_dir_skip, f'{f_name}_Y.xlsx'))
        print(f'Number of skip train data: {len(df_X)}')
    
    #break

-------------------------
0_2914_106510_122129909_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106510, 'CategoryId': 122129909}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
2_3616_106515_150581531_29.xlsx
{'MarketId': 2, 'AdvertiserId': 3616, 'CampaignId': 106515, 'CategoryId': 150581531}
Number of consistent train data: 2
-------------------------
3_2773_157502_587672534_94.xlsx
{'MarketId': 3, 'AdvertiserId': 2773, 'CampaignId': 157502, 'CategoryId': 587672534}
-------------------------
1_3051_188627_1_13.xlsx
{'MarketId': 1, 'AdvertiserId': 3051, 'CampaignId': 188627, 'CategoryId': 1}
-------------------------
0_2914_106509_1884256507_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106509, 'CategoryId': 1884256507}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
0_2914_106510_1884256507_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106510, 'Categ

{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106509, 'CategoryId': 1191599194}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
2_3616_106515_1403161255_27.xlsx
{'MarketId': 2, 'AdvertiserId': 3616, 'CampaignId': 106515, 'CategoryId': 1403161255}
Number of consistent train data: 2
-------------------------
0_2914_106510_1191599194_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106510, 'CategoryId': 1191599194}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
4_22864_106511_1654090720_604.xlsx
{'MarketId': 4, 'AdvertiserId': 22864, 'CampaignId': 106511, 'CategoryId': 1654090720}
Number of consistent train data: 540
Number of skip train data: 476
-------------------------
1_3051_105355_739735262_698.xlsx
{'MarketId': 1, 'AdvertiserId': 3051, 'CampaignId': 105355, 'CategoryId': 739735262}
Number of consistent train data: 679
Number of skip train data: 662
------------------------

Number of consistent train data: 686
Number of skip train data: 664
-------------------------
4_22864_106511_1019498501_696.xlsx
{'MarketId': 4, 'AdvertiserId': 22864, 'CampaignId': 106511, 'CategoryId': 1019498501}
Number of consistent train data: 670
Number of skip train data: 651
-------------------------
0_2914_106509_647500945_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106509, 'CategoryId': 647500945}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
0_2914_106509_1048444253_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106509, 'CategoryId': 1048444253}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
0_2914_106509_393836642_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106509, 'CategoryId': 393836642}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
2_15912_106516_1899268006_699.xlsx
{'MarketId': 2, '

Number of consistent train data: 686
Number of skip train data: 664
-------------------------
0_2914_106510_1816201462_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106510, 'CategoryId': 1816201462}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
0_2914_106509_1816201462_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106509, 'CategoryId': 1816201462}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
4_22864_106511_1345039788_687.xlsx
{'MarketId': 4, 'AdvertiserId': 22864, 'CampaignId': 106511, 'CategoryId': 1345039788}
Number of consistent train data: 661
Number of skip train data: 617
-------------------------
0_2914_106510_1600325479_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106510, 'CategoryId': 1600325479}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
1_3051_105356_1714381509_253.xlsx
{'MarketId': 1

Number of consistent train data: 686
Number of skip train data: 664
-------------------------
2_15911_106512_2026840770_699.xlsx
{'MarketId': 2, 'AdvertiserId': 15911, 'CampaignId': 106512, 'CategoryId': 2026840770}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
1_3051_105356_676054762_253.xlsx
{'MarketId': 1, 'AdvertiserId': 3051, 'CampaignId': 105356, 'CategoryId': 676054762}
Number of consistent train data: 240
Number of skip train data: 218
-------------------------
1_3051_105356_272007381_253.xlsx
{'MarketId': 1, 'AdvertiserId': 3051, 'CampaignId': 105356, 'CategoryId': 272007381}
Number of consistent train data: 240
Number of skip train data: 218
-------------------------
0_2914_106510_739735262_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106510, 'CategoryId': 739735262}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
2_15912_106516_1467760103_699.xlsx
{'MarketId': 2, 'Ad

{'MarketId': 3, 'AdvertiserId': 2773, 'CampaignId': 157502, 'CategoryId': 1749236875}
-------------------------
2_15911_106512_1771391024_699.xlsx
{'MarketId': 2, 'AdvertiserId': 15911, 'CampaignId': 106512, 'CategoryId': 1771391024}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
2_15911_106512_1933655103_675.xlsx
{'MarketId': 2, 'AdvertiserId': 15911, 'CampaignId': 106512, 'CategoryId': 1933655103}
Number of consistent train data: 596
Number of skip train data: 568
-------------------------
2_3616_106515_2123310757_23.xlsx
{'MarketId': 2, 'AdvertiserId': 3616, 'CampaignId': 106515, 'CategoryId': 2123310757}
Number of consistent train data: 1
-------------------------
1_3051_105356_603041281_254.xlsx
{'MarketId': 1, 'AdvertiserId': 3051, 'CampaignId': 105356, 'CategoryId': 603041281}
Number of consistent train data: 241
Number of skip train data: 219
-------------------------
0_2914_106510_1108209194_699.xlsx
{'MarketId': 0, 'AdvertiserId'

Number of consistent train data: 686
Number of skip train data: 664
-------------------------
1_3051_157505_1007561482_93.xlsx
{'MarketId': 1, 'AdvertiserId': 3051, 'CampaignId': 157505, 'CategoryId': 1007561482}
-------------------------
0_2914_106510_647500945_699.xlsx
{'MarketId': 0, 'AdvertiserId': 2914, 'CampaignId': 106510, 'CategoryId': 647500945}
Number of consistent train data: 686
Number of skip train data: 664
-------------------------
1_3051_157505_1884256507_93.xlsx
{'MarketId': 1, 'AdvertiserId': 3051, 'CampaignId': 157505, 'CategoryId': 1884256507}


### テストデータを分割
- test_1: 2019/12/4までを学習して2019/12/5~2019/12/11の予測
- test_2: 2019/12/11までを学習して2019/12/12~2019/12/18の予測

In [7]:
def split(train_X_end, train_Y_end, \
          val_X_begin, val_Y_begin, \
          val_X_end, val_Y_end, \
          train_dir, output_dir, \
          in_dir='./data/two_years_each/data'):
    
    df_X_train_all, df_Y_train_all = [], []
    df_X_val_all, df_Y_val_all = [], []

    for i, f in enumerate(os.listdir(in_dir)):

        #if i > 2: break
        f_name = os.path.splitext(f)[0]
        if not os.path.exists(os.path.join(train_dir, f'{f_name}_X.xlsx')):
            print(f'not found: {f_name}')
            continue

        df_X = pd.read_excel(os.path.join(train_dir, f'{f_name}_X.xlsx'), header=0, index_col=0)
        df_Y = pd.read_excel(os.path.join(train_dir, f'{f_name}_Y.xlsx'), header=0, index_col=0)

        assert len(df_X) == len(df_Y), '[Error] length of X and Y do not match !'

        df_X_train, df_X_val = df_X[:train_X_end], df_X[val_X_begin:val_X_end]
        df_Y_train, df_Y_val = df_Y[:train_Y_end], df_Y[val_Y_begin:val_Y_end]

        assert len(df_X_train) == len(df_Y_train) and len(df_X_val) == len(df_Y_val), '[Error] length of X and Y do not match !'

        df_X_train_all += [df_X_train]
        df_Y_train_all += [df_Y_train]
        df_X_val_all += [df_X_val]
        df_Y_val_all += [df_Y_val]

        #break

    df_X_train_all, df_Y_train_all = pd.concat(df_X_train_all), pd.concat(df_Y_train_all)
    df_X_val_all, df_Y_val_all = pd.concat(df_X_val_all), pd.concat(df_Y_val_all)

    df_X_train_all.to_excel(os.path.join(output_dir, 'train_X.xlsx'))
    df_Y_train_all.to_excel(os.path.join(output_dir, 'train_Y.xlsx'))
    df_X_val_all.to_excel(os.path.join(output_dir, 'val_X.xlsx'))
    df_Y_val_all.to_excel(os.path.join(output_dir, 'val_Y.xlsx'))

In [8]:
# test_1
train_X_end, train_Y_end = '2019-11-27', '2019-12-04'
val_X_begin, val_Y_begin = '2019-11-28', '2019-12-05'
val_X_end, val_Y_end = '2019-12-04', '2019-12-11'

for task in ['skip', 'consistent']:
    train_dir = f'./data/two_years_each/data_{task}/'
    output_dir = f'./data/two_years_each/test_1/{task}/'

    split(train_X_end, train_Y_end, 
          val_X_begin, val_Y_begin,
          val_X_end, val_Y_end,
          train_dir, output_dir)

not found: 2_3616_106515_150581531_29
not found: 3_2773_157502_587672534_94
not found: 1_3051_188627_1_13
not found: 3_2773_106513_-1_620
not found: 2_3616_106515_1251677358_26
not found: 2_3616_106515_0_24
not found: 2_15911_106512_0_699
not found: 2_3616_106515_559598750_24
not found: 3_2773_157502_477150441_95
not found: 0_2914_106510_0_699
not found: 1_3051_157505_782042272_93
not found: 3_2773_157502_647396435_94
not found: 3_2773_157502_800963742_95
not found: 2_3616_106515_2118878002_25
not found: 1_3051_188627_0_48
not found: 1_3051_105356_0_254
not found: 2_3616_106515_537842849_23
not found: 0_2914_106509_0_699
not found: 2_3616_106515_1403161255_27
not found: 3_2773_157502_357928842_94
not found: 3_2773_157502_739735262_94
not found: 3_2773_157502_307427601_94
not found: 4_22864_106511_0_690
not found: 2_3616_106515_1129747985_23
not found: 3_2773_157502_874499868_99
not found: 1_3051_188627_-1_25
not found: 1_3051_105355_-1_52
not found: 3_2773_157502_1881376968_97
not foun

In [9]:
# test_2
train_X_end, train_Y_end = '2019-12-04', '2019-12-11'
val_X_begin, val_Y_begin = '2019-12-05', '2019-12-12'
val_X_end, val_Y_end = '2019-12-11', '2019-12-18'

for task in ['skip', 'consistent']:
    train_dir = f'./data/two_years_each/data_{task}/'
    output_dir = f'./data/two_years_each/test_2/{task}/'

    split(train_X_end, train_Y_end, 
          val_X_begin, val_Y_begin,
          val_X_end, val_Y_end,
          train_dir, output_dir)

not found: 2_3616_106515_150581531_29
not found: 3_2773_157502_587672534_94
not found: 1_3051_188627_1_13
not found: 3_2773_106513_-1_620
not found: 2_3616_106515_1251677358_26
not found: 2_3616_106515_0_24
not found: 2_15911_106512_0_699
not found: 2_3616_106515_559598750_24
not found: 3_2773_157502_477150441_95
not found: 0_2914_106510_0_699
not found: 1_3051_157505_782042272_93
not found: 3_2773_157502_647396435_94
not found: 3_2773_157502_800963742_95
not found: 2_3616_106515_2118878002_25
not found: 1_3051_188627_0_48
not found: 1_3051_105356_0_254
not found: 2_3616_106515_537842849_23
not found: 0_2914_106509_0_699
not found: 2_3616_106515_1403161255_27
not found: 3_2773_157502_357928842_94
not found: 3_2773_157502_739735262_94
not found: 3_2773_157502_307427601_94
not found: 4_22864_106511_0_690
not found: 2_3616_106515_1129747985_23
not found: 3_2773_157502_874499868_99
not found: 1_3051_188627_-1_25
not found: 1_3051_105355_-1_52
not found: 3_2773_157502_1881376968_97
not foun

In [10]:
# val
train_X_end, train_Y_end = '2019-10-28', '2019-11-04'
val_X_begin, val_Y_begin = '2019-10-29', '2019-11-05'
val_X_end, val_Y_end = '2019-11-27', '2019-12-04'

for task in ['skip', 'consistent']:
    train_dir = f'./data/two_years_each/data_{task}/'
    output_dir = f'./data/two_years_each/val/{task}/'

    split(train_X_end, train_Y_end, 
          val_X_begin, val_Y_begin,
          val_X_end, val_Y_end,
          train_dir, output_dir)

not found: 2_3616_106515_150581531_29
not found: 3_2773_157502_587672534_94
not found: 1_3051_188627_1_13
not found: 3_2773_106513_-1_620
not found: 2_3616_106515_1251677358_26
not found: 2_3616_106515_0_24
not found: 2_15911_106512_0_699
not found: 2_3616_106515_559598750_24
not found: 3_2773_157502_477150441_95
not found: 0_2914_106510_0_699
not found: 1_3051_157505_782042272_93
not found: 3_2773_157502_647396435_94
not found: 3_2773_157502_800963742_95
not found: 2_3616_106515_2118878002_25
not found: 1_3051_188627_0_48
not found: 1_3051_105356_0_254
not found: 2_3616_106515_537842849_23
not found: 0_2914_106509_0_699
not found: 2_3616_106515_1403161255_27
not found: 3_2773_157502_357928842_94
not found: 3_2773_157502_739735262_94
not found: 3_2773_157502_307427601_94
not found: 4_22864_106511_0_690
not found: 2_3616_106515_1129747985_23
not found: 3_2773_157502_874499868_99
not found: 1_3051_188627_-1_25
not found: 1_3051_105355_-1_52
not found: 3_2773_157502_1881376968_97
not foun